In [12]:
!pip install transformers

In [13]:
import pandas as pd
from transformers import BertTokenizer

tokenizer=BertTokenizer.from_pretrained("bert-base-uncased") #Bert-base의 토크나이저

In [14]:
#Here is the sentence I want embeddings for. -> 토큰화 실습
result=tokenizer.tokenize('Here is the sentence I want embeddings for.')
print(result)

['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']


In [15]:
#BERT단어 집합에 특정 단어가 있는지 조회 : .vocab[]
print(tokenizer.vocab['here'])

2182


In [16]:
print(tokenizer.vocab['em'])

7861


In [17]:
print(tokenizer.vocab['##bed'])

8270


In [18]:
#BERT의 단어 집합을 vocabulary.txt에 저장
with open('vocabulary.txt','w') as f :
  for token in tokenizer.vocab.keys():
    f.write(token+'\n')

In [19]:
df=pd.read_fwf('vocabulary.txt',header=None)
df

,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


In [20]:
print('단어 집합의 크기 :',len(df))

단어 집합의 크기 : 30522


In [21]:
#데이터프레임의 인덱스가 해당 단어와 맵핑된 정수
#정수로부터 맵핑된 단어 확인 가능
#4667번 단어 확인
df.loc[4667].values[0]

'##ding'

In [22]:
df.loc[102].values[0]

'[SEP]'

## 구글 BERT의 마스크드 언어 모델 (Masked Language Model) 실습

### 마스크드 언어 모델과 토크나이저

In [23]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [24]:
#BERT를 마스크드 언어 모델 형태로 로드
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")


All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [29]:
#BERT의 입력
inputs = tokenizer('Soccer is a really fun [MASK].', return_tensors='tf')

In [30]:
#input_ids : 정수 인코딩 결과 확인
print(inputs['input_ids'])

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


In [31]:
#token_type_ids : 세그먼트 인코딩 결과 확인
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


In [32]:
#실제 단어와 패딩 토큰을 구분하는 용도인 어텐션 마스크 확인
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


### [MASK] 토큰 예측하기
- FillMaskPipeline : 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서 보여줌

In [33]:
from transformers import FillMaskPipeline
pip=FillMaskPipeline(model=model, tokenizer=tokenizer)

In [34]:
#[MASK]의 위치에 들어갈 수 있는 상위 5개의 후보 단어 출력
pip('Soccer is a really fun [MASK].')

[{'score': 0.7621122598648071,
  'token': 4368,
  'token_str': 'sport',
  'sequence': 'soccer is a really fun sport.'},
 {'score': 0.20341946184635162,
  'token': 2208,
  'token_str': 'game',
  'sequence': 'soccer is a really fun game.'},
 {'score': 0.012208531610667706,
  'token': 2518,
  'token_str': 'thing',
  'sequence': 'soccer is a really fun thing.'},
 {'score': 0.0018630254780873656,
  'token': 4023,
  'token_str': 'activity',
  'sequence': 'soccer is a really fun activity.'},
 {'score': 0.0013354856055229902,
  'token': 2492,
  'token_str': 'field',
  'sequence': 'soccer is a really fun field.'}]

In [35]:
pip('The Avengers is a really fun [MASK].')


[{'score': 0.256289541721344,
  'token': 2265,
  'token_str': 'show',
  'sequence': 'the avengers is a really fun show.'},
 {'score': 0.17284125089645386,
  'token': 3185,
  'token_str': 'movie',
  'sequence': 'the avengers is a really fun movie.'},
 {'score': 0.11107686907052994,
  'token': 2466,
  'token_str': 'story',
  'sequence': 'the avengers is a really fun story.'},
 {'score': 0.0724899172782898,
  'token': 2186,
  'token_str': 'series',
  'sequence': 'the avengers is a really fun series.'},
 {'score': 0.07046637684106827,
  'token': 2143,
  'token_str': 'film',
  'sequence': 'the avengers is a really fun film.'}]

## 한국어 BERT의 마스크드 언어 모델 (Masked Language Model) 실습

In [37]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [39]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


### BERT의 입력

In [40]:
inputs=tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')

In [42]:
print(inputs['input_ids']) #정수 인코딩 결과 확인

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [43]:
print(inputs['token_type_ids']) #세그먼트 인코딩 결과 확인

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


In [44]:
print(inputs['attention_mask']) #어텐션 마스크 확인

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


### [MASK] 토큰 예측하기

In [45]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [46]:
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.8963507413864136,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.025957578793168068,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033943690359592,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.00792438630014658,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844219915568829,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [47]:
pip('어벤져스는 정말 재미있는 [MASK]다.')

[{'score': 0.838240921497345,
  'token': 3771,
  'token_str': '영화',
  'sequence': '어벤져스는 정말 재미있는 영화 다.'},
 {'score': 0.028275633230805397,
  'token': 568,
  'token_str': '거',
  'sequence': '어벤져스는 정말 재미있는 거 다.'},
 {'score': 0.01718932017683983,
  'token': 4665,
  'token_str': '드라마',
  'sequence': '어벤져스는 정말 재미있는 드라마 다.'},
 {'score': 0.01498968992382288,
  'token': 3758,
  'token_str': '이야기',
  'sequence': '어벤져스는 정말 재미있는 이야기 다.'},
 {'score': 0.009382679127156734,
  'token': 4938,
  'token_str': '장소',
  'sequence': '어벤져스는 정말 재미있는 장소 다.'}]

## 구글 BERT의 다음 문장 예측 (Next Sentence Prediction)

In [48]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [49]:
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [50]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."


In [51]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')


In [52]:
print(encoding['input_ids'])

tf.Tensor(
[[  101  1999  3304  1010 10733  2366  1999  5337 10906  1010  2107  2004
   2012  1037  4825  1010  2003  3591  4895 14540  6610  2094  1012   102
  10733  2003  8828  2007  1996  2224  1997  1037  5442  1998  9292  1012
   1999 10017 10906  1010  2174  1010  2009  2003  3013  2046 17632  2015
   2000  2022  8828  2096  2218  1999  1996  2192  1012   102]], shape=(1, 58), dtype=int32)


In [53]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)


[CLS] : 101
[SEP] : 102


In [54]:
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


In [56]:
print(encoding['token_type_ids']) #0과 1로 두 개의 문장을 구분

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 58), dtype=int32)


### 다음 문장 예측하기

In [57]:
logits=model(encoding['input_ids'],token_type_ids=encoding['token_type_ids'])[0]
softmax=tf.keras.layers.Softmax()
probs=softmax(logits)
print(probs)

tf.Tensor([[9.9999714e-01 2.8381912e-06]], shape=(1, 2), dtype=float32)


In [58]:
print('최종 예측 레이블 :',tf.math.argmax(probs,axis=-1).numpy())

최종 예측 레이블 : [0]


In [59]:
# 상관없는 두 개의 문장
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]


## 한국어 BERT의 다음 문장 예측

In [60]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [61]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [62]:
# 이어지는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 레이블 : [0]


In [63]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 레이블 : [1]
